In [81]:
import dask.dataframe as dd

In [ ]:
train= dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\train.csv',
    dtype={
        'id': 'int64',
        'store_nbr': 'int64',
        'item_nbr': 'int64',
        'unit_sales': 'float32',
        'onpromotion': 'Int8'
    },
    parse_dates=['date']
)
items=dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\items.csv',
    dtype={
        'item_nbr':'int64',
        'family':'category',
        'class':'int64',
        'perishable':'Int8'
    }
    )
oil=dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\oil.csv',
    dtype={
        'dcoilwtico':'float32'
    },
    parse_dates=['date']
    )
stores=dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\stores.csv',
    dtype={
        'store_nbr':'int64',
        'city':'category',
        'state':'category',
        'type':'category',
        'cluster':'int64'
    }
    )
transactions=dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\transactions.csv',
    dtype={
        'store_nbr':'int64',
        'transactions':'int64'
    },
    parse_dates=['date']
    )
weather=dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\ecuador_weather.csv',
    dtype={
        'temperature':'float32',
        'precipitation':'float32',
        'city':'category'
    },
    parse_dates=['date']
    )
holidays=dd.read_csv(
    r'C:\Users\alqay\OneDrive\Desktop\GitHub Rebos\-Sales-Forecasting-and-Demand-Prediction\Data\store-sales-time-series-forecasting\holidays_events.csv',
    dtype={
        'type':'category',
        'locale':'category',
        'locale_name':'category',
        'transferred':'boolean'
    },
    parse_dates=['date'],
    usecols=["type", "locale", "locale_name","transferred","date"]
    )

In [84]:
holidays = holidays[~holidays['transferred']]

In [85]:
holidays = holidays.drop(columns='transferred')

In [86]:
df=train.merge(stores,on='store_nbr',how='left')\
        .merge(items,on='item_nbr',how='left')\
        .merge(transactions,on=['date','store_nbr'],how='left')\
        .merge(oil,on='date',how='left')\
        .merge(holidays,on='date',how='left')\
        .merge(weather,on=['date','city'],how='left')

In [87]:
df

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type_x,cluster,family,class,perishable,transactions,dcoilwtico,type_y,locale,locale_name,temperature,precipitation
npartitions=78,,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],int64,int64,float64,float64,string,string,string,int64,string,int64,int64,int64,float64,string,string,string,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [88]:
df=df.drop(columns=['id','item_nbr','cluster','class'])

In [89]:
df

,date,store_nbr,unit_sales,onpromotion,city,state,type_x,family,perishable,transactions,dcoilwtico,type_y,locale,locale_name,temperature,precipitation
npartitions=78,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float64,float64,string,string,string,string,int64,int64,float64,string,string,string,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [90]:
df=df.rename(columns={'type_x':'store_type','dcoilwtico':'oil_price','type_y':'day_type'})

In [91]:
df

,date,store_nbr,unit_sales,onpromotion,city,state,store_type,family,perishable,transactions,oil_price,day_type,locale,locale_name,temperature,precipitation
npartitions=78,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float64,float64,string,string,string,string,int64,int64,float64,string,string,string,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [92]:
agg_type={
    'unit_sales':'sum',
    'onpromotion':'mean',
    'city':'first',
    'state':'first',
    'store_type':'first',
    'perishable':'mean',
    'transactions':'sum',
    'oil_price':'first',
    'day_type':'first',
    'locale':'first',
    'locale_name':'first',
    'temperature':'first',
    'precipitation':'first'
}
df=df.groupby(['date','store_nbr','family']).agg(agg_type).reset_index()

In [93]:
print(df)

Dask DataFrame Structure:
                         date store_nbr  family unit_sales onpromotion    city   state store_type perishable transactions oil_price day_type  locale locale_name temperature precipitation
npartitions=1                                                                                                                                                                             
               datetime64[ns]     int64  string    float64     float64  string  string     string    float64        int64   float64   string  string      string     float64       float64
                          ...       ...     ...        ...         ...     ...     ...        ...        ...          ...       ...      ...     ...         ...         ...           ...
Dask Name: reset_index, 28 expressions
Expr=ResetIndex(frame=GroupbyAggregation(frame=RenameFrame(frame=Drop(frame=Merge(7e1caf1), columns=['id', 'item_nbr', 'cluster', 'class']), columns={'type_x': 'store_type', 'dcoilwtico':

In [94]:
df['onpromotion']=(df['onpromotion'] > .5).astype('int8')
df['perishable']=(df['perishable'] > .5).astype('int8')

In [95]:
df

,date,store_nbr,family,unit_sales,onpromotion,city,state,store_type,perishable,transactions,oil_price,day_type,locale,locale_name,temperature,precipitation
npartitions=1,,,,,,,,,,,,,,,,
,datetime64[ns],int64,string,float64,int8,string,string,string,int8,int64,float64,string,string,string,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [96]:
for col in df.columns:
    print(f"{col}: {df[col].dtype}")


date: datetime64[ns]
store_nbr: int64
family: string
unit_sales: float64
onpromotion: int8
city: string
state: string
store_type: string
perishable: int8
transactions: int64
oil_price: float64
day_type: string
locale: string
locale_name: string
temperature: float64
precipitation: float64


In [97]:
df.to_parquet(r"E:\my data\sales_All_data_final_grouped.parquet", engine="pyarrow")